# Assessing performance efficiency of SAINT transformer architecture with structured tabular data for wildfire prediction

In [4]:
# Install dependencies
%pip install torch pytorch_lightning scikit-learn pandas numpy matplotlib

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/3f/14/e105b8ef6d324e789c1589e95cb0ab63f3e07c2216d68b1178b7c21b7d2a/torch-2.2.2-cp311-none-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for pytorch_lightning from https://files.pythonhosted.org/packages/de/a9/e14821cfaf08e8d78185cca0477c9d3a62bafe1b4b530100f7b66bb1f7bb/pytorch_lightning-2.5.1.post0-py3-none-any.whl.metadata
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/f5/bd/af9f655456f60fe1d575f54fb14704ee299b16e999704817a7645dfce6b0/matplotlib-3.10.3-cp311-cp311-macosx_10_12_x86_64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/4d/36/2a115987e2d8c300a974597416d9de88f2444426de9571f4b59b2cca3acc/filelock-3.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/a2/09/77d55d46fd61b4a135c444fc97158ef34a095e56

In [5]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('dataset.csv')

# Display the first few rows
print(df.head())

  system:index  EVH   EVT    NDVI  PRES_max  SPFH_max    TMP_max  WIND_max  \
0        0_0_0   75  3092  5691.0   97165.0  0.006520  17.850000      4.88   
1        0_1_0   89  3900  6176.0   98114.0  0.006578  21.589990      4.59   
2        0_2_0   96  3097  6563.0   97881.0  0.007021  19.989984      3.85   
3        0_3_0   21  3296  4034.0   98933.0  0.006326  22.369989      4.08   
4        0_4_0   99  3097  8027.0   97243.0  0.007369  16.290002      5.66   

   burned        date  elevation  sm_profile  sm_profile_wetness  sm_rootzone  \
0       0  2023-04-01  348.75693    0.365752            0.797881     0.389533   
1       0  2023-04-01  363.53400    0.356276            0.777190     0.380164   
2       0  2023-04-01  409.15347    0.361830            0.789306     0.381324   
3       0  2023-04-01  263.18298    0.358382            0.781829     0.382392   
4       0  2023-04-01  323.89856    0.367979            0.802734     0.387794   

   sm_rootzone_wetness  sm_surface  sm_surfa